In [ ]:
import glob
from pathlib import Path

from blooms_ml.utils import prepare_data

In [ ]:
files_dia = sorted(glob.glob(
    f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_dia/*dia*.nc"
))[:10]
files_avg = sorted(glob.glob(
    f"{Path.home()}/fram_shmiak/ROHO800_hindcast_2007_2019_v2bu/roho800_v2bu_avg/*avg*.nc"
))[:10]

In [ ]:
df = prepare_data(files_dia, files_avg)

In [ ]:
df

In [ ]:
df.ocean_time.plot()